#           1.导入库

In [1]:
# encoding: UTF-8
import re
import os
import numpy as np
import scipy.io as sio
from Bio import SeqIO
import tensorflow as tf
from time import time
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

# 2.定义CNN的共享函数

In [ ]:
#定义权值
def weight(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.01), name='weight')

#定义偏置
def bias(shape):
    return tf.Variable(tf.constant(0.01, shape=shape), name='bias')

#定义卷积操作
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='VALID')

#定义最大池化操作
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

#定义平均池化操作
def avg_pool_2x2(x):
    return tf.nn.avg_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')


## 3.定义卷积神经网络
  - 返回测试集预测结果

In [ ]:
def cnn(x_train, x_test, y_train, y_test):
    X_TRAIN_SIZE = int(x_train.shape[0])
    TRAIN_EPOCHES = 50
    BATCH_SIZE = 100
    TOTAL_BATCH = int( np.ceil( X_TRAIN_SIZE / BATCH_SIZE))
    
    x_train = np.reshape(x_train,[-1,31,21,1])
    x_test = np .reshape(x_test,[-1,31,21,1])
    #输入层 31-by-21
    with tf.name_scope('input_layer'):
        x = tf.placeholder(tf.float32, shape=[None, 31, 21, 1], name='x')

    #第1个卷积层
    with tf.name_scope('conv_1'):
        W1 = weight([21,21,1,128])
        b1 = bias([128])
        conv_1 = conv2d(x, W1) + b1
        conv_2 = tf.nn.relu(conv_1)

    #第1个池化层 16-by-11
    with tf.name_scope('pool_1'):
        pool_1 = max_pool_2x2(conv_1)

#     #第2个卷积层
#     with tf.name_scope('conv_2'):
#         W2 = weight([5,5,32,64])
#         b2 = bias([64])
#         conv_2 = conv2d(pool_1, W2) + b2
#         conv_2 = tf.nn.relu(conv_2)

#     #第2个池化层 8-by-6
#     with tf.name_scope("pool_2"):
#         pool_2 = max_pool_2x2(conv_2)

#     #第3个卷积层
#     with tf.name_scope('conv_3'):
#         W3 = weight([3,3,64,128])
#         b3 = bias([128])
#         conv_3 = conv2d(pool_2, W3) + b3
#         conv_3 = tf.nn.relu(conv_3)

#     #第3个池化层 4-by-3
#     with tf.name_scope('pool_3'):
#         pool_3 = max_pool_2x2(conv_3)

    #全连接层
    with tf.name_scope('fc'):
        #将最后一个池化层的128个通道的4-by-3的图像转换为一维向量，长度是128*4*3=1536
#         W4 = weight([1536,256]) #全连接层定义256个神经元
#         b4 = bias([256])
#         flat = tf.reshape(pool_3, [-1, 1536])
#         h = tf.nn.relu(tf.matmul(flat, W4)) + b4
        
        W4 = weight([6*1*128,256]) #全连接层定义256个神经元
        b4 = bias([256])
        flat = tf.reshape(pool_1, [-1, 6*1*128])
        h = tf.nn.relu(tf.matmul(flat, W4)) + b4
        
        keep_prob = tf.placeholder(tf.float32)
        h_dropout = tf.nn.dropout(h, keep_prob)

    #输出层
    with tf.name_scope('output_layer'):
        W5 = weight([256,2])
        b5 = bias([2])
        pred = tf.nn.softmax(tf.matmul(h_dropout, W5) + b5)
    
    #构建网络模型
    with tf.name_scope("optimizer"):
        #定义占位符
        y = tf.placeholder(tf.int32, shape=[None, 2], name="label")
        #定义损失函数
        loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,
                                                                                 labels=y))
        
        #选择优化器
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss_function)
    
    #定义准确率
    with tf.name_scope("evalulation"):
        correct_prediction = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    #训练模型
    epoch= tf.Variable(0, name='epoch', trainable=False)
    STARTTIME = time()
    start_epoch = 0
    
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        # 设置检查点存储目录
        ckpt_dir = "log/"
        if not os.path.exists(ckpt_dir):
            os.makedirs(ckpt_dir)
        #生成saver
        saver = tf.train.Saver(max_to_keep=5)
        # 创建 summary_writer，用于写图文件
        summary_writer = tf.summary.FileWriter(ckpt_dir, sess.graph)
        # 如果有检查点文件，读取最新的检查点文件，恢复各种变量值
        ckpt = tf.train.latest_checkpoint(ckpt_dir )
        if ckpt != None:
            saver.restore(sess, ckpt)     
        else:
            print("Training from scratch.")

        start_epoch= sess.run(epoch)
        print("Training starts form {} epoch.".format(start_epoch+1))
        
        #迭代训练
        for ep in range(start_epoch, start_epoch + TRAIN_EPOCHES):
            x_train, y_train = shuffle(x_train, y_train)
            for i in range(TOTAL_BATCH):
                for i in range(a):

                    batch_x = names['X_train_%s' % i]
                    batch_y = names['Y_train_%s' % i]
                    
                    sess.run(optimizer,feed_dict={x: batch_x, y: batch_y, keep_prob:0.5})
                    if i % 100 == 0:
                        print("Step {}".format(i), "finished")

            loss,acc = sess.run([loss_function,accuracy],feed_dict={x:names['X_test_%s' % i], y:names['Y_test_%s' % i], keep_prob:1.0})
            
            print("Train epoch:", '%02d' % (sess.run(epoch)+1), \
                  "Loss=","{:.6f}".format(loss)," Accuracy=",acc)

            #保存检查点
            #saver.save(sess,ckpt_dir+"DBPSite_cnn_model.cpkt",global_step=ep+1)

            sess.run(epoch.assign(ep+1))
    
        duration =time()-STARTTIME
        print("Train finished takes:",duration)   
    
        #计算测试集上的预测结果
        y_pred = sess.run(pred, feed_dict={x:X_test, y:Y_test, keep_prob:1.0})
        
    return y_pred

## 4.使用K-Fold交叉验证

In [ ]:
#对数据集X按KFold划分训练集和测试集
def splitByKFold(X, n=5):
    x_trains = []
    x_tests = []
    kf = KFold(n_splits=n)
    for train_index, test_index in kf.split(X):
        x_train, x_test = X[train_index], X[test_index]
        x_trains.append(x_train)
        x_tests.append(x_test)
     
    return x_trains, x_tests

#把正类样本集和负类样本集分别做K-Fold划分训练集和测试集
#其中正类样本集是少数类集，对其做上采样
#然后把对应的正类样本和负类样本保存在名为"fold_k.npz"的文件中
#fold_k.npz文件中保存以下数据：
#    traindata: 训练样本集 
#    testdata: 测试样本集
#    traintarget: 训练样本分类标签
#    testtarget: 测试样本分类标签
def saveKFoldSMoteData(positive_data, negative_data, n=5):
    for i in range(5):
        x_trains_pos, x_tests_pos = splitByKFold(positive_data, n)
        x_trains_neg, x_tests_neg = splitByKFold(negative_data, n)
        x_train_pos = x_trains_pos[i] #正类训练数据
        x_train_neg = x_trains_neg[i] #负类训练数据
        x_test_pos = x_tests_pos[i] #正类测试数据
        x_test_neg = x_tests_neg[i] #负类测试数据

        x_train_pos = SMote(x_train_pos) #正类训练数据是少数类样本，进行上采样

        k_train_pos = int( len( x_train_pos)) #正类训练样本数
        k_train_neg = int( len( x_train_neg)) #负类训练样本数
        k_test_pos = int( len( x_test_pos)) #正类测试样本数
        k_test_neg = int( len( x_test_neg)) #负类测试样本数

        y_train_pos = np.tile([1,0], (k_train_pos,1))
        y_train_neg = np.tile([0,1], (k_train_neg,1))
        y_test_pos = np.tile([1,0], (k_test_pos,1))
        y_test_neg = np.tile([0,1], (k_test_neg,1))

        x_train = np.append(x_train_pos, x_train_neg, axis=0)
        y_train = np.append(y_train_pos, y_train_neg, axis=0)

        x_test = np.append(x_test_pos, x_test_neg, axis=0)
        y_test = np.append(y_test_pos, y_test_neg, axis=0)

        x_train = x_train.astype(np.float32)
        x_test = x_test.astype(np.float32)

        filename = 'fold_' + str(i) + '.npz'
        np.savez(filename, traindata=x_train, testdata=x_test, traintarget=y_train, testtarget=y_test)
        
def validationKFold(n=5):      
    y_logists = np.ndarray((0,2))
    y_preds = np.ndarray((0,2))
    for i in range(n):
        filename =  'fold_' + str(i) + '.npz'
        D = np.load(filename)
        x_train = D['traindata']
        x_test = D['testdata']
        y_train = D['traintarget']
        y_test = D['testtarget']
        y_pred = cnn(x_train, x_test, y_train, y_test)
        
        y_logists = np.append(y_logists, y_test, axis=0)
        y_preds = np.append(y_preds, y_pred, axis=0)
        
    return y_logists, y_preds

## 5.计算预测率、混淆矩阵

In [ ]:
#对数据集X按KFold划分训练集和测试集
def splitByKFold(X, n=5):
    x_trains = []
    x_tests = []
    kf = KFold(n_splits=n)
    for train_index, test_index in kf.split(X):
        x_train, x_test = X[train_index], X[test_index]
        x_trains.append(x_train)
        x_tests.append(x_test)
     
    return x_trains, x_tests

#把正类样本集和负类样本集分别做K-Fold划分训练集和测试集
#其中正类样本集是少数类集，对其做上采样
#然后把对应的正类样本和负类样本保存在名为"fold_k.npz"的文件中
#fold_k.npz文件中保存以下数据：
#    traindata: 训练样本集 
#    testdata: 测试样本集
#    traintarget: 训练样本分类标签
#    testtarget: 测试样本分类标签
def saveKFoldSMoteData(positive_data, negative_data, n=5):
    for i in range(5):
        x_trains_pos, x_tests_pos = splitByKFold(positive_data, n)
        x_trains_neg, x_tests_neg = splitByKFold(negative_data, n)
        x_train_pos = x_trains_pos[i] #正类训练数据
        x_train_neg = x_trains_neg[i] #负类训练数据
        x_test_pos = x_tests_pos[i] #正类测试数据
        x_test_neg = x_tests_neg[i] #负类测试数据

        x_train_pos = SMote(x_train_pos) #正类训练数据是少数类样本，进行上采样

        k_train_pos = int( len( x_train_pos)) #正类训练样本数
        k_train_neg = int( len( x_train_neg)) #负类训练样本数
        k_test_pos = int( len( x_test_pos)) #正类测试样本数
        k_test_neg = int( len( x_test_neg)) #负类测试样本数

        y_train_pos = np.tile([1,0], (k_train_pos,1))
        y_train_neg = np.tile([0,1], (k_train_neg,1))
        y_test_pos = np.tile([1,0], (k_test_pos,1))
        y_test_neg = np.tile([0,1], (k_test_neg,1))

        x_train = np.append(x_train_pos, x_train_neg, axis=0)
        y_train = np.append(y_train_pos, y_train_neg, axis=0)

        x_test = np.append(x_test_pos, x_test_neg, axis=0)
        y_test = np.append(y_test_pos, y_test_neg, axis=0)

        x_train = x_train.astype(np.float32)
        x_test = x_test.astype(np.float32)

        filename = 'test_fold_' + str(i) + '.npz'
        np.savez(filename, traindata=x_train, testdata=x_test, traintarget=y_train, testtarget=y_test)
        
def validationKFold(n=5):      
    y_logists = np.ndarray((0,2))
    y_preds = np.ndarray((0,2))
    for i in range(n):
        filename =  'test_fold_' + str(i) + '.npz'
        D = np.load(filename)
        x_train = D['traindata']
        x_test = D['testdata']
        y_train = D['traintarget']
        y_test = D['testtarget']
        y_pred = cnn(x_train, x_test, y_train, y_test)
        
        y_logists = np.append(y_logists, y_test, axis=0)
        y_preds = np.append(y_preds, y_pred, axis=0)
        
    return y_logists, y_preds

## 6.计算预测率、混淆矩阵

In [ ]:
# 计算预测的准确率
def evalution(y_logists, y_preds):
    correct_prediction = tf.equal(tf.argmax(y_preds, 1),
                                      tf.argmax(y_logists, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    with tf.Session() as sess:
        acc = sess.run(accuracy)
    
    return acc

# 绘制混淆矩阵的函数
# 参数1  cm 混淆矩阵中显示的数值 二维数组
# 参数2 cmap 混淆矩阵中的颜色
# 参数3 title 标题
def plot_confusion_matrix(cm, classes, title='混淆矩阵', cmap=plt.cm.Greens):
    # imshow() 表示绘制并显示二维图 有18个参数
    # 参数1 X 混淆矩阵中显示的数值 二维数组
    # 参数2 cmap 颜色 plt.cm.Blues表示蓝色 plt.cm.Reds表示红色 plt.cm.Greens表示绿色
    # 参数5 interpolation 插值法 一般有如下值
    #     nearest 最近邻插值法
    #     bilinear 双线性插值法
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['axes.unicode_minus'] = False
    plt.imshow(cm, cmap=cmap, interpolation="nearest")
    plt.title(title)  # 标题
    plt.colorbar()  # 显示颜色的进度条
    tick_marks = np.arange(2)  # [0 1]
    plt.xticks(tick_marks, classes)  # 对x轴上分类进行标记
    plt.yticks(tick_marks, classes)  # 对y轴上分类进行标记

    thresh = np.mean(cm)
    for i in range(2):
        for j in range(2):
            plt.text(i, j, cm[j][i],
                     horizontalalignment='center',
                     color='white' if cm[i][j] >= thresh else 'black')

    plt.xlabel('预测值')
    plt.ylabel('真实值')

## 7.主函数（保存处理后的数据）

In [ ]:
%matplotlib inline
#establishBenchmarkOneHot('..\\data\\PDNA-224.fasta','..\\data\\PDNA-224-seqs-OneHot.mat')
#splitWindows(15, '..\\data\\PDNA-224-seqs-OneHot.mat', '..\\data\\PDNA-224-binding-sites.txt', '..\\data\\PDNA-224-OneHot-15.mat')
# bindingsite = sio.loadmat('data\\PDNA-224-OneHot-15.mat')
# negative_data = bindingsite['negative']
# positive_data = bindingsite['positive']

negative_data = sio.loadmat('data\\PDNA-224-ONEHOT-11-N.mat')
positive_data = sio.loadmat('data\\PDNA-224-ONEHOT-11-P.mat')
n_X = N_data['n_data']
n_Y = N_data['n_target']
p_X = P_data['p_data']
p_Y = P_data['p_target']

#随机打乱正样本数据并返回
N_p = p_X.shape[0]
print("N_p \t",N_p)
indx = list(range(N_p))
random.shuffle(indx)
p_x=p_X[indx]
p_y=p_Y[indx]


#随机打乱负样本数据并返回
N_n = n_X.shape[0]
print("N_n \t",N_n)
indx = list(range(N_n))
random.shuffle(indx)
n_x=n_X[indx]
n_y=n_Y[indx]




#从正负样本中各抽出800条数据，得到测试集 1600个


X_test = np.vstack((n_X[52770:53570],p_X[2978:3778]))
Y_test = np.vstack((n_Y[52770:53570],p_Y[2978:3778]))
#随机打乱测试集数据并返回
N_test = X_test.shape[0]
print("N_test \t",N_test)
indx = list(range(N_test))
random.shuffle(indx)
X_test=X_test[indx]
Y_test=Y_test[indx]
X_test = X_test.reshape(len(X_test),-1)
Y_test = Y_test.reshape(len(Y_test),-1)




#处理训练集，正样本数量不变，在剩余52770个负样本中抽取3022个不放回，组成6000条数据为一组的训练集

NN = len(n_X[0:52770])
print(NN)

#负样本批次大小
N_size = 3022
#计算一共有负样本多少个批次
NN_batch = NN // N_size

#print(NN_batch)
a=0
#得到17个数据集，正样本不变，负样本从'PDNA-224-ONEHOT-11-N.mat中抽出3822个不放回
for i in range(NN_batch):
               
        if (i+2)*N_size > NN: #  53570/3022=17.46
                X = np.vstack((n_X[i*N_size:NN],p_X[0:2978]))
                Y = np.vstack((n_Y[i*N_size:NN],p_Y[0:2978]))
        else:
                X = np.vstack((n_X[i*N_size:(i+1)*N_size],p_X[0:2978]))
                Y = np.vstack((n_Y[i*N_size:(i+1)*N_size],p_Y[0:2978]))
                #print(i*N_size)              
        #print(len(X),len(Y))
        #print('')
        
        X = X.reshape(len(X),-1)
        Y = Y.reshape(len(Y),-1) 
        #print(X)
        #loo = LeaveOneOut()
        kf = KFold(n_splits=5)
        kf.get_n_splits(X)
        for train_index, test_index in kf.split(X):           
            #得到85个数据集，保存在动态变量中,例如 X_train_1
            
            names['X_train_%s' % a],names['X_test_%s' % a]=X[train_index], X[test_index]
            names['Y_train_%s' % a],names['Y_test_%s' % a]=Y[train_index], Y[test_index]
            a+=1 

y_pred = cnn(x_train, x_test, y_train, y_test)

#saveKFoldSMoteData(positive_data, negative_data, 5)

# 8.主函数（训练，测试并画出混淆矩阵）

In [ ]:
#对数据进行训练
y_logists, y_preds = validationKFold(5)
acc = evalution(y_logists, y_preds)
print("5-Fold prediction accuracy=%.4f" % acc)


In [ ]:
#混淆矩阵
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    #print(y_logists)
    
    a=tf.argmax(y_logists,1)
    b=tf.argmax(y_preds,1)
    xx=sess.run(a)
    yy=sess.run(b)
    print(xx,yy)
    #cnf_matrix = confusion_matrix(y_logists[:,0], y_preds[:,0])
    cnf_matrix = confusion_matrix(xx, yy)
    print(cnf_matrix)
    recall = cnf_matrix[0][0] / (cnf_matrix[0][0] + cnf_matrix[0][1])
    print('recall: ', recall)
    plot_confusion_matrix(cnf_matrix, [1, 0], cmap=plt.cm.Reds)

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
PDNA-onehot-测试集和训练集分开-混淆矩阵

"""




names = locals()
#load benchmark dataset
N_data = sio.loadmat('data\\PDNA-224-ONEHOT-11-N.mat')
P_data = sio.loadmat('data\\PDNA-224-ONEHOT-11-P.mat')
n_X = N_data['n_data']
n_Y = N_data['n_target']
p_X = P_data['p_data']
p_Y = P_data['p_target']



#随机打乱正样本数据并返回
N_p = p_X.shape[0]
print("N_p \t",N_p)
indx = list(range(N_p))
random.shuffle(indx)
p_x=p_X[indx]
p_y=p_Y[indx]


#随机打乱负样本数据并返回
N_n = n_X.shape[0]
print("N_n \t",N_n)
indx = list(range(N_n))
random.shuffle(indx)
n_x=n_X[indx]
n_y=n_Y[indx]





# 得到测试集 1600个

X_test = np.vstack((n_X[52770:53570],p_X[2978:3778]))
Y_test = np.vstack((n_Y[52770:53570],p_Y[2978:3778]))
#随机打乱测试集数据并返回
N_test = X_test.shape[0]
print("N_test \t",N_test)
indx = list(range(N_test))
random.shuffle(indx)
X_test=X_test[indx]
Y_test=Y_test[indx]
X_test = X_test.reshape(len(X_test),-1)
Y_test = Y_test.reshape(len(Y_test),-1)




#处理训练集，正样本数量不变，在剩余52770个负样本中抽取3022个不放回，组成6000条数据为一组的训练集

NN = len(n_X[0:52770])
print(NN)

#负样本批次大小
N_size = 3022
#计算一共有负样本多少个批次
NN_batch = NN // N_size

#print(NN_batch)
a=0
#得到17个数据集，正样本不变，负样本从'PDNA-224-ONEHOT-11-N.mat中抽出3822个不放回
for i in range(NN_batch):
               
        if (i+2)*N_size > NN: #  53570/3022=17.46
                X = np.vstack((n_X[i*N_size:NN],p_X[0:2978]))
                Y = np.vstack((n_Y[i*N_size:NN],p_Y[0:2978]))
        else:
                X = np.vstack((n_X[i*N_size:(i+1)*N_size],p_X[0:2978]))
                Y = np.vstack((n_Y[i*N_size:(i+1)*N_size],p_Y[0:2978]))
                #print(i*N_size)              
        #print(len(X),len(Y))
        #print('')
        
        X = X.reshape(len(X),-1)
        Y = Y.reshape(len(Y),-1) 
        #print(X)
        #loo = LeaveOneOut()
        kf = KFold(n_splits=5)
        kf.get_n_splits(X)
        for train_index, test_index in kf.split(X):           
            #得到85个数据集，保存在动态变量中,例如 X_train_1
            
            names['X_train_%s' % a],names['X_test_%s' % a]=X[train_index], X[test_index]
            names['Y_train_%s' % a],names['Y_test_%s' % a]=Y[train_index], Y[test_index]
            a+=1 
            
y_pred = cnn(x_train, x_test, y_train, y_test)            
  